In [ ]:
import os
from dotenv import load_dotenv

MONGO_DB_URI = os.getenv("MONGO_DB_URI")

In [ ]:
# %pip install llama-index-vector-stores-mongodb

In [ ]:
import HFH

In [ ]:
import pymongo
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext

mongodb_client = pymongo.MongoClient(MONGO_DB_URI)
atlas_vector_search = MongoDBAtlasVectorSearch(
    mongodb_client,
    db_name = "llamaindex_db",
    collection_name = "test",
    index_name = "vector_index"
)
storage_context = StorageContext.from_defaults(vector_store=atlas_vector_search)

In [ ]:
####  Create embedding and store into database

# from llama_index.core import SimpleDirectoryReader
# from llama_index.core import VectorStoreIndex,load_index_from_storage


# docs = SimpleDirectoryReader("./data/test/").load_data()
# print(docs)
# vector_store_index = VectorStoreIndex.from_documents(
#     documents=docs, storage_context=storage_context, show_progress=True
# )

In [ ]:
#### Load embedding from database and create query retriver

from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_vector_store(atlas_vector_search)

In [ ]:
retriever = index.as_retriever(similarity_top_k=3)
nodes = retriever.retrieve("what is iris?")

for node in nodes:
    print(node)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import PromptTemplate

# Instantiate Atlas Vector Search as a retriever
vector_store_retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

query_engine = index.as_query_engine(
    text_qa_template= PromptTemplate("""
        Below is an instruction that describes a task.     
        Write a response that appropriately completes the request.\n\n    
        ### Instruction:\n{query_str}\n\n### Response:""")
)

In [ ]:
query_engine.query("which flowers are found below 2200m height")


In [ ]:
# print("\nSource documents: ")
# print(response.source_nodes)